## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

#### Best Practises
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW,TFIDF,Word2vec
4. Train ML algorithms

In [2]:
# Load the dataset
import pandas as pd
data=pd.read_csv('Kindle Reviews/all_kindle_review.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
df=data[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [4]:
df.shape

(12000, 2)

In [5]:
## Missing Values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [7]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [85]:
## Preprocessing And Cleaning

In [8]:
## postive review is 1 and negative review is 0
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

/tmp/ipykernel_39207/3208188810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)


In [9]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [10]:
## 1. Lower All the cases
df['reviewText']=df['reviewText'].str.lower()

/tmp/ipykernel_39207/1654308119.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


In [11]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [12]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from bs4 import BeautifulSoup

In [14]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


/tmp/ipykernel_39207/1658462958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
/tmp/ipykernel_39207/1658462958.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
/tmp/ipykernel_39207/1658462958.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [15]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [16]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

In [17]:
lemmatizer=WordNetLemmatizer()

In [18]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...


True

In [22]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

/tmp/ipykernel_39207/1959687590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))


In [23]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [24]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],
                                              test_size=0.20)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [106]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [107]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [108]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [109]:
y_pred_bow=nb_model_bow.predict(X_test_bow)

In [110]:
y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

In [113]:
confusion_matrix(y_test,y_pred_bow)

array([[511, 308],
       [692, 889]], dtype=int64)

In [112]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.5833333333333334


In [115]:
confusion_matrix(y_test,y_pred_tfidf)

array([[502, 317],
       [687, 894]], dtype=int64)

In [114]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.5816666666666667


# Word2vec

In [27]:
from gensim.utils import simple_preprocess

In [29]:
def sent_to_words(sentences):
  return simple_preprocess(sentences)

In [63]:
# df_copy = df.copy(deep=True)
df = df_copy.copy(deep=True)

In [64]:
df['reviewText']=df['reviewText'].apply(lambda x:sent_to_words(x))

In [65]:
import gensim

In [66]:
## Lets train Word2vec from scratch
model=gensim.models.Word2Vec(df['reviewText'])

In [67]:
## To Get All the Vocabulary
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'series',
 'well',
 'much',
 'first',
 'even',
 'didnt',
 'short',
 'know',
 'way',
 'great',
 'could',
 'make',
 'sex',
 'little',
 'dont',
 'two',
 'thing',
 'want',
 'think',
 'find',
 'plot',
 'romance',
 'also',
 'end',
 'life',
 'im',
 'see',
 'enjoyed',
 'go',
 'scene',
 'never',
 'written',
 'take',
 'woman',
 'many',
 'lot',
 'kindle',
 'year',
 'say',
 'thought',
 'work',
 'bit',
 'found',
 'going',
 'give',
 'interesting',
 'liked',
 'writing',
 'novel',
 'loved',
 'another',
 'feel',
 'better',
 'got',
 'come',
 'man',
 'hot',
 'still',
 'back',
 'enough',
 'though',
 'people',
 'star',
 'reader',
 'made',
 'something',
 'review',
 'part',
 'friend',
 'page',
 'cant',
 'bad',
 'world',
 'need',
 'free',
 'new',
 'keep',
 'wasnt',
 'doesnt',
 'relationship',
 'enjoy',
 'recommend',
 'together',
 'next',
 'felt',
 'start',
 'best',
 'put',

In [68]:
model.corpus_count

12000

In [69]:
model.epochs

5

In [70]:
model.wv.similar_by_word('good')

[('decent', 0.8423567414283752),
 ('dissappointed', 0.8396615982055664),
 ('nice', 0.8349616527557373),
 ('great', 0.824540376663208),
 ('linethis', 0.8151394724845886),
 ('abouti', 0.810543954372406),
 ('easy', 0.8063785433769226),
 ('amazing', 0.8020614981651306),
 ('satisfying', 0.797497034072876),
 ('overall', 0.7954015135765076)]

In [71]:
model.wv['good'].shape

(100,)

In [42]:
jj=0
for word in df['reviewText']:
  
  print(word)
  jj+=1
  if jj==2:
    break

['jace', 'rankin', 'may', 'short', 'he', 'nothing', 'mess', 'man', 'hauled', 'saloon', 'undertaker', 'know', 'he', 'famous', 'bounty', 'hunter', 'oregon', 'shot', 'man', 'saloon', 'finished', 'year', 'long', 'quest', 'avenge', 'sister', 'murder', 'trying', 'figure', 'next', 'snotty', 'nosed', 'farm', 'boy', 'rescued', 'gang', 'bully', 'offer', 'money', 'kill', 'man', 'forced', 'ranch', 'reluctantly', 'agrees', 'bring', 'man', 'justice', 'kill', 'outright', 'first', 'need', 'tell', 'sister', 'widower', 'newskyla', 'kyle', 'springer', 'bailey', 'riding', 'trail', 'sleeping', 'ground', 'past', 'month', 'trying', 'find', 'jace', 'want', 'revenge', 'man', 'killed', 'husband', 'took', 'ranch', 'amongst', 'crime', 'shes', 'keen', 'detour', 'jace', 'want', 'take', 'realizes', 'shes', 'option', 'hide', 'behind', 'boy', 'persona', 'best', 'try', 'keep', 'pace', 'confrontation', 'along', 'way', 'get', 'shot', 'jace', 'discovers', 'kyles', 'kyla', 'come', 'clean', 'whole', 'reason', 'need', 'scoun

In [48]:
df['reviewText'][0]

['jace',
 'rankin',
 'may',
 'short',
 'he',
 'nothing',
 'mess',
 'man',
 'hauled',
 'saloon',
 'undertaker',
 'know',
 'he',
 'famous',
 'bounty',
 'hunter',
 'oregon',
 'shot',
 'man',
 'saloon',
 'finished',
 'year',
 'long',
 'quest',
 'avenge',
 'sister',
 'murder',
 'trying',
 'figure',
 'next',
 'snotty',
 'nosed',
 'farm',
 'boy',
 'rescued',
 'gang',
 'bully',
 'offer',
 'money',
 'kill',
 'man',
 'forced',
 'ranch',
 'reluctantly',
 'agrees',
 'bring',
 'man',
 'justice',
 'kill',
 'outright',
 'first',
 'need',
 'tell',
 'sister',
 'widower',
 'newskyla',
 'kyle',
 'springer',
 'bailey',
 'riding',
 'trail',
 'sleeping',
 'ground',
 'past',
 'month',
 'trying',
 'find',
 'jace',
 'want',
 'revenge',
 'man',
 'killed',
 'husband',
 'took',
 'ranch',
 'amongst',
 'crime',
 'shes',
 'keen',
 'detour',
 'jace',
 'want',
 'take',
 'realizes',
 'shes',
 'option',
 'hide',
 'behind',
 'boy',
 'persona',
 'best',
 'try',
 'keep',
 'pace',
 'confrontation',
 'along',
 'way',
 'get',

In [72]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [73]:
df.reset_index(inplace=True,drop=True)

In [74]:
from tqdm import tqdm

In [75]:
#apply for the entire sentences
import numpy as np
X=[]
for i in tqdm(range(df['reviewText'].shape[0])):
    X.append(avg_word2vec(df['reviewText'][i]))

100%|██████████| 12000/12000 [00:22<00:00, 535.33it/s]


In [76]:
X[1].shape

(100,)

In [77]:
## this is the final independent features
df_final=pd.DataFrame()
for i in range(0,len(X)):
    df_final=pd.concat([df_final,pd.DataFrame(X[i].reshape(1,-1))],ignore_index=True)

In [78]:
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.190876,0.299696,0.066805,0.198568,0.010939,-0.376994,0.106371,0.597527,-0.157722,-0.144998,...,0.457667,0.141552,-0.082826,0.066569,0.586887,0.246513,-0.114409,-0.257017,-0.040106,0.011918
1,-0.242568,0.367649,-0.225217,0.298501,0.290761,-0.651012,0.272120,0.692607,-0.235799,-0.188970,...,0.808119,0.135443,-0.296261,-0.186605,0.307821,0.313666,0.073976,-0.044009,0.039976,-0.066954
2,-0.171993,0.373183,0.025252,0.111245,0.187294,-0.616652,0.258597,0.727480,-0.247095,-0.224593,...,0.604365,0.130770,-0.115537,-0.195722,0.399303,0.287619,0.138317,-0.083470,-0.147506,0.091659
3,-0.291258,0.265636,0.063058,0.127512,0.179630,-0.670693,0.220088,0.760358,-0.106182,-0.042850,...,0.547217,0.034943,-0.004863,-0.371276,0.136672,0.198750,0.541872,0.156884,-0.188941,0.039407
4,0.020799,0.509025,0.009858,-0.038542,0.063837,-0.599369,0.249485,0.793739,-0.098094,-0.298830,...,0.741200,0.203457,-0.113877,-0.358183,0.476779,0.185091,0.118499,-0.270365,0.032973,-0.079921


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewText  12000 non-null  object
 1   rating      12000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 187.6+ KB


In [81]:
df.head(2)

,reviewText,rating
0,"[jace, rankin, may, short, he, nothing, mess, ...",1
1,"[great, short, read, didnt, want, put, read, o...",1


In [82]:
y=df['rating']

In [85]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_final,y,test_size=0.20)

In [86]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
4676,-0.266296,0.283300,0.012932,0.165826,0.174820,-0.652993,0.141922,0.710845,-0.240382,-0.210382,...,0.623360,0.062331,-0.285555,-0.346827,0.191300,0.289358,0.338823,0.157020,-0.203745,0.225536
8954,-0.266061,0.387412,-0.038476,0.178902,0.076737,-0.584242,0.320859,0.724838,-0.187432,-0.193446,...,0.752328,0.060876,-0.214368,-0.198626,0.351879,0.210175,0.092494,-0.081357,-0.000431,-0.029432
6938,-0.184813,0.257894,-0.068779,0.107411,0.304182,-0.475897,0.131903,0.762374,-0.206590,-0.260169,...,0.543561,0.178399,-0.123968,-0.195313,0.511688,0.198624,0.201033,-0.190101,0.003586,0.001896
357,-0.138230,0.405680,0.050288,0.172374,0.048482,-0.545738,0.237537,0.666867,-0.119819,-0.319544,...,0.639613,0.046023,-0.132851,-0.099866,0.315950,0.158021,0.050432,-0.114023,-0.002542,-0.083919
7391,-0.133320,0.271689,-0.099034,0.293610,0.213905,-0.684629,0.341782,0.740359,-0.093608,-0.428012,...,0.742238,0.084030,-0.348732,-0.408752,0.346665,0.293103,0.160134,0.010881,-0.281442,0.033217


In [87]:
y_train

4676    1
8954    1
6938    1
357     0
7391    1
       ..
749     1
4398    0
6212    1
987     1
8194    0
Name: rating, Length: 9600, dtype: int64

In [88]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [89]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [90]:
y_pred=classifier.predict(X_test)

In [92]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.765


In [93]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.68      0.56      0.62       806
           1       0.80      0.87      0.83      1594

    accuracy                           0.77      2400
   macro avg       0.74      0.71      0.72      2400
weighted avg       0.76      0.77      0.76      2400

